In [1]:
import pandas as pd
df = pd.read_pickle('dataset_with_hadm.pkl')
df.head()

,SUBJECT_ID,HADM_ID,ADMISSION_DAYS,GENDER,AGE,50801,50802,50803,50804,50805,...,G11,G12,G13,G14,G15,G16,G17,G18,G19,G20
0,22,165315,2,0,65,462.0,0.0,24.0,22.0,1.0,...,0,0,0,0,0,0,0,1,1,0
1,23,152223,6,1,71,462.0,0.0,24.0,26.0,1.0,...,0,0,0,0,0,0,0,0,0,1
2,23,124321,7,1,75,462.0,1.0,24.0,27.0,1.0,...,0,0,0,0,0,1,0,0,0,1
3,24,161859,3,1,39,462.0,0.0,24.0,27.0,1.0,...,0,0,0,0,0,0,0,0,0,0
4,25,129635,4,1,59,462.0,0.0,24.0,26.0,1.0,...,0,0,0,0,0,0,0,0,0,0


In [2]:
df_nohadm = df.drop(['HADM_ID','SUBJECT_ID'], axis=1)

In [4]:
x = df_nohadm.iloc[:2000,:483]
y = df_nohadm.iloc[:2000,483:]
print(x.columns,y.columns)

Index(['ADMISSION_DAYS',         'GENDER',            'AGE',            50801,
                  50802,            50803,            50804,            50805,
                  50806,            50808,
       ...
                  51512,            51513,            51514,            51515,
                  51516,            51517,            51518,            51519,
                  51529,            51532],
      dtype='object', length=483) Index(['G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'G7', 'G8', 'G9', 'G10', 'G11',
       'G12', 'G13', 'G14', 'G15', 'G16', 'G17', 'G18', 'G19', 'G20'],
      dtype='object')


In [5]:
seed = 15
from sklearn.model_selection import train_test_split
#x_tra,x_test,y_tra,y_test = train_test_split(x,y,test_size=0.25,random_state=seed)
x_train,x_val,y_train,y_val = train_test_split(x,y,test_size=0.25,random_state=seed) 
# i wanna have train,val,test ya cool

In [6]:
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb
from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import ClassifierChain
from sklearn.model_selection import GridSearchCV,StratifiedKFold
from sklearn.metrics import jaccard_score,roc_auc_score,confusion_matrix,hamming_loss


In [7]:
n_folds = 5

def aucroc(y_val,y_pred):
    #kf = KFold(n_folds, shuffle=True, random_state=seed).get_n_splits(x_train.values)
    auc = roc_auc_score(y_val, y_pred)
    return(auc)

In [20]:
class average_models(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
   
    # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]  #len(list)=4
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
        
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)*y.shape[1])) # rows*4
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, y_pred.shape[1]] = y_pred    # we need to mostly change here only
                
        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
   
    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)

In [21]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB 
gnb = OneVsRestClassifier(GaussianNB()) 
xgb_model = OneVsRestClassifier(XGBClassifier(30,learning_rate=0.05,n_jobs=-1)) #make it lesser..
lgbm_model = OneVsRestClassifier(LGBMClassifier(30,learning_rate=0.05,n_jobs=-1))
cat_model = OneVsRestClassifier(CatBoostClassifier(30,learning_rate=0.05))
rf = OneVsRestClassifier(RandomForestClassifier(20,n_jobs=-1))
# eh? simply i tried lol some thought just flashed that i should maybe try this haan but u didnt fit lol
#the others also arent fitting oops

In [22]:
import numpy as np
avg_models = average_models(base_models = (xgb_model, cat_model,gnb), meta_model = rf) 
avg_models.fit(x_train, y_train)
stacked_train_pred = avg_models.predict(x_train).reshape(-1,1)
stacked_pred = avg_models.predict(x_val).reshape(-1,1)
print(aucroc(y_train,stacked_train_pred))
print(aucroc(y_val,stacked_pred))

KeyError: "None of [Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    9,   10,\n            ...\n            1485, 1486, 1488, 1490, 1492, 1493, 1496, 1497, 1498, 1499],\n           dtype='int64', length=1200)] are in the [columns]"